In [2]:
!pip install symengine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 17.7 MB/s eta 0:00:00


In [3]:


import math
import sympy as sp
import numpy as np
import symengine




In [6]:
# ===== Functions =====

def define_symbols():
    theta1, theta2, theta3 = sp.symbols('theta1 theta2 theta3')
    return theta1, theta2, theta3

def forward_kinematics(theta, link_lengths=[0.4, 0.3, 0.2]):
    """
    Compute symbolic forward kinematics for a 3-DOF planar arm.

    Args:
        theta: tuple/list of symbolic joint variables (theta1, theta2, theta3)
        link_lengths: list of link lengths [L1, L2, L3]

    Returns:
        xf, yf: symbolic expressions for end-effector position
    """
    L1, L2, L3 = link_lengths
    theta1, theta2, theta3 = theta

    xf = L1*sp.cos(theta1) + L2*sp.cos(theta2) + L3*sp.cos(theta3)
    yf = L1*sp.sin(theta1) + L2*sp.sin(theta2) + L3*sp.sin(theta3)

    return xf, yf

def evaluate_fk(xf, yf, angles):
    """Substitute numeric angles into forward kinematics expressions."""
    xf_val = xf.subs(angles).evalf()
    yf_val = yf.subs(angles).evalf()
    return xf_val, yf_val

def compute_jacobian(xf, yf, theta):
    """Compute symbolic Jacobian matrix."""
    return sp.Matrix([xf, yf]).jacobian(theta)

def evaluate_jacobian(J, angles):
    """Evaluate symbolic Jacobian at given numeric angles."""
    return np.array(J.subs(angles)).astype(np.float64)

def symbolic_error(xf, yf, target_x, target_y):
    """Define a symbolic error function for potential optimization."""
    return ((xf - target_x)**2 + (yf - target_y)**2)/2

# ===== Gradient Descent =====
def gradient_descent(xf, yf, theta, target, lr=0.1, iterations=100):
    """Perform gradient descent to move end-effector toward target (X, Y)."""
    theta_vals = np.array([math.pi/4, math.pi/4, math.pi/4], dtype=float)  # initial angles
    X_target, Y_target = target

    # Convert symbolic error to derivatives
    E = symbolic_error(xf, yf, X_target, Y_target)
    grads = [sp.lambdify(theta, sp.diff(E, t)) for t in theta]

    for i in range(iterations):
        # Compute gradients
        grad_vals = np.array([g(*theta_vals) for g in grads], dtype=float)
        # Update angles
        theta_vals -= lr * grad_vals

        # Compute current error
        E_val = E.subs({theta[j]: theta_vals[j] for j in range(3)}).evalf()
        print(f"Iter {i+1}: theta = {theta_vals}, error = {E_val}")

    return theta_vals


In [10]:
# ===== Main Script =====
if __name__ == "__main__":
    # Define symbolic variables
    theta1, theta2, theta3 = define_symbols()
    theta = (theta1, theta2, theta3)

    # Forward kinematics
    xf, yf = forward_kinematics(theta)
    print("Symbolic Forward Kinematics:")
    print("Xf:", xf)
    print("Yf:", yf)

    # Numeric evaluation
    angles = {theta1: math.pi/2, theta2: math.pi/6, theta3: math.pi/9}
    xf_val, yf_val = evaluate_fk(xf, yf, angles)
    print("\nNumeric FK at given angles:")
    print("Xf =", xf_val)
    print("Yf =", yf_val)

    # Jacobian
    J = compute_jacobian(xf, yf, theta)
    print("\nSymbolic Jacobian:")
    sp.pprint(J)

    J_val = evaluate_jacobian(J, angles)
    print("\nJacobian as NumPy array at given angles:")
    print(J_val)

    # Symbolic error function (example)
    target_x, target_y = 0.8, 0.5
    E = symbolic_error(xf, yf, target_x, target_y)
    print("\nSymbolic Error function E:")
    print(E)

    # Optional: SymEngine example
    if symengine:
        print("\nUsing SymEngine for Jacobian (optional)")
        x, y, z = symengine.symbols('x y z')
        f = symengine.sympify([
            '0.4*cos(x) + 0.3*cos(y) + 0.2*cos(z)',
            '0.4*sin(x) + 0.3*sin(y) + 0.2*sin(z)'
        ])
        J_se = symengine.zeros(len(f), 3)
        for i, fi in enumerate(f):
            for j, s in enumerate([x, y, z]):
                J_se[i, j] = symengine.diff(fi, s)
        print("SymEngine Jacobian matrix:")
        print(J_se)
    # Target position
    target = (0.8, 0.5)
    theta_final = gradient_descent(xf, yf, theta, target, lr=0.1, iterations=10)
    print("\nFinal joint angles after gradient descent:", theta_final)


Symbolic Forward Kinematics:
Xf: 0.4*cos(theta1) + 0.3*cos(theta2) + 0.2*cos(theta3)
Yf: 0.4*sin(theta1) + 0.3*sin(theta2) + 0.2*sin(theta3)

Numeric FK at given angles:
Xf = 0.447746145292513
Yf = 0.618404028665134

Symbolic Jacobian:
⎡-0.4⋅sin(θ₁)  -0.3⋅sin(θ₂)  -0.2⋅sin(θ₃)⎤
⎢                                        ⎥
⎣0.4⋅cos(θ₁)   0.3⋅cos(θ₂)   0.2⋅cos(θ₃) ⎦

Jacobian as NumPy array at given angles:
[[-4.00000000e-01 -1.50000000e-01 -6.84040287e-02]
 [ 2.44929360e-17  2.59807621e-01  1.87938524e-01]]

Symbolic Error function E:
0.125*(0.8*sin(theta1) + 0.6*sin(theta2) + 0.4*sin(theta3) - 1)**2 + 0.32*(0.5*cos(theta1) + 0.375*cos(theta2) + 0.25*cos(theta3) - 1)**2

Using SymEngine for Jacobian (optional)
SymEngine Jacobian matrix:
[-0.4*sin(x), -0.3*sin(y), -0.2*sin(z)]
[0.4*cos(x), 0.3*cos(y), 0.2*cos(z)]

Iter 1: theta = [0.77691288 0.7790342  0.78115552], error = 0.0213994288705491
Iter 2: theta = [0.76868051 0.7728586  0.77703759], error = 0.0201890165864022
Iter 3: theta = [0.7